In [1]:
# !pip install evaluate datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.


In [2]:
import os
import torch
import evaluate
import numpy as np
from datasets import load_dataset, Dataset, Audio
from transformers import ASTFeatureExtractor, ASTForAudioClassification, ASTConfig, Trainer, TrainingArguments

In [3]:
DATASET_NAME = ''
CACHE_DIR = './cache'
ROOT_DIR = './'

# --- Audio Processing Parameters ---
TARGET_SAMPLE_RATE = 16000  # Hz (16kHz)
CHUNK_LENGTH_MS = 1000      # milliseconds (1 second)
CHUNK_LENGTH_SAMPLES = int(TARGET_SAMPLE_RATE * CHUNK_LENGTH_MS / 1000)

# --- Training Hyperparameters ---
BATCH_SIZE = 32
EPOCHS = 5
LR = 3e-5

# --- Model Configuration ---
MODEL_CHECKPOINT = "MIT/ast-finetuned-audioset-10-10-0.4593"
MODEL_SAVE_DIR = os.path.join(ROOT_DIR, 'output_models')
CHECKPOINT_FILENAME = "ast_best_model"

In [4]:
def load_dataset_splits(dataset_name: str) -> Dataset:
    """Load a dataset from the Hugging Face Hub.

    Args:
        dataset_name: Name of the dataset to load

    Returns:
        Hugging Face Dataset object
    """
    try:
        dataset = load_dataset(dataset_name, cache_dir=CACHE_DIR)
        dataset = dataset.cast_column("audio", Audio(sampling_rate=TARGET_SAMPLE_RATE, mono=True))

        print(f"Dataset loaded successfully with splits: {list(dataset.keys())}")
        print(f"Train split size: {dataset['train'].num_rows}")
        if 'valid' in dataset:
            print(f"Validation split size: {dataset['valid'].num_rows}")
        if 'test' in dataset:
            print(f"Test split size: {dataset['test'].num_rows}")

        return dataset
    except Exception as e:
        print(f"Failed to load dataset {dataset_name}: {e}")
        raise

In [5]:
def build_transformer_model(num_classes: int, model_checkpoint: str):
    """
    Loads a pre-trained Audio Spectrogram Transformer (AST) model
    from Hugging Face for PyTorch.

    Args:
        num_classes (int): The number of output classes for the classification layer.
        model_checkpoint (str): The Hugging Face AST model identifier.

    Returns:
        torch.nn.Module: The PyTorch AST model with a potentially resized classification head.
    """
    # Load the pre-trained AST model
    # Set ignore_mismatched_sizes=True to allow replacing the classification head
    # if the number of classes differs from the pre-trained model.
    label = {'0': 0, '1': 1}

    ast_config = ASTConfig.from_pretrained(model_checkpoint)

    ast_config.num_labels = num_classes
    ast_config.label2id = label
    ast_config.id2label = {v: k for k, v in label.items()}

    model = ASTForAudioClassification.from_pretrained(
        model_checkpoint,
        cache_dir=CACHE_DIR,
        config=ast_config,
        ignore_mismatched_sizes=True, # Allows replacing the classification head
    )
    return model

def get_feature_extractor(model_checkpoint: str):
    """
    Loads the AST feature extractor.

    Args:
        model_checkpoint (str): The Hugging Face AST model identifier.

    Returns:
        ASTFeatureExtractor: The AST feature extractor instance.
    """
    # Load the feature extractor
    try:
        feature_extractor = ASTFeatureExtractor.from_pretrained(
            model_checkpoint,
            sampling_rate=TARGET_SAMPLE_RATE
        )
        return feature_extractor
    except Exception as e:
        print(f"Error loading feature extractor: {e}")
        raise

In [ ]:
try:
    feature_extractor = get_feature_extractor(MODEL_CHECKPOINT)
    print(f"Successfully loaded feature extractor: {feature_extractor}")
    print(f"Target sampling rate from feature extractor: {feature_extractor.sampling_rate} Hz")
    print(f"Feature extractor expects max_length: {feature_extractor.max_length} samples")
except Exception as e:
    print(f"Failed to load feature extractor: {e}", exc_info=True)
    feature_extractor = None

In [7]:
def preprocess_features(example):
    """Applies the AST feature extractor to a batch of audio data.

    This function is designed to be used with `dataset.map()`.

    Args:
        batch: A dictionary representing a batch of examples from the HF dataset.
        Expected to contain an 'audio' key with audio data.

    Returns:
        A dictionary containing the processed 'input_values'.
    """
    if feature_extractor is None:
        raise RuntimeError("Feature extractor was not loaded successfully. Cannot preprocess.")

    # Ensure audio data is in the expected format (list of numpy arrays)
    audio_arrays = [x["array"] for x in example['input_values']]

    # Apply the feature extractor
    # It handles resampling (if needed, though ideally done already),
    # spectrogram computation, normalization, padding, and truncation.
    inputs = feature_extractor(
        audio_arrays,
        sampling_rate=feature_extractor.sampling_rate, # Use the actual rate from the batch
        max_length=CHUNK_LENGTH_SAMPLES,
        padding='max_length',
        truncation=True,
        return_tensors="pt" # Return PyTorch tensors suitable for PyTorch
    )

    # The feature extractor typically returns a dictionary.
    # The key for the processed features is often 'input_values'.
    # For AST, it's usually 'input_values'. Let's rename it for clarity if needed.
    # Check the output keys of your specific extractor if unsure.
    if "input_values" in inputs:
        example["input_values"] = inputs["input_values"]
    else:
        print(f"Feature extractor output did not contain expected keys ('input_values'). Found: {inputs.keys()}")
        # Handle error appropriately, maybe return None or raise exception
        raise KeyError("Could not find processed features in feature extractor output.")

    # Ensure the label is present and correctly formatted (e.g., integer index)
    # This assumes labels are already numerical indices in the dataset
    if 'label' not in example:
        raise KeyError(f"Label column 'label' not found in the batch.")

    return example

In [8]:
def get_device():
    if torch.cuda.is_available():
        print("CUDA available. Using GPU.")
        return torch.device("cuda")
    else:
        print("CUDA/MPS not available. Using CPU.")
        return torch.device("cpu")

DEVICE = get_device()

CUDA available. Using GPU.


In [ ]:
ds = load_dataset_splits(dataset_name=DATASET_NAME)

Dataset loaded successfully with splits: ['train', 'valid', 'test']
Train split size: 585180
Validation split size: 32510
Test split size: 32511

In [10]:
ds = ds.rename_column('audio', 'input_values')
processed_datasets = ds.with_transform(preprocess_features)

In [ ]:
model = build_transformer_model(num_classes=2, model_checkpoint=MODEL_CHECKPOINT)
model.to(DEVICE)

In [16]:
args = TrainingArguments(
    output_dir="test-train-model",
    eval_strategy = "epoch",
    save_strategy = "epoch",
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    learning_rate=LR,
    logging_steps=10,
    gradient_accumulation_steps=4,
    warmup_ratio=0.1,
    metric_for_best_model="accuracy",
    load_best_model_at_end=True,
    fp16=True,
    disable_tqdm=False,
    # push_to_hub=True,
    # tpu_num_cores=48
)


In [17]:
accuracy = evaluate.load("accuracy")
recall = evaluate.load("recall")
precision = evaluate.load("precision")
f1 = evaluate.load("f1")

AVERAGE = "binary"

def compute_metrics(eval_pred):
    logits = eval_pred.predictions
    predictions = np.argmax(logits, axis=1)
    metrics = accuracy.compute(predictions=predictions, references=eval_pred.label_ids)
    metrics.update(precision.compute(predictions=predictions, references=eval_pred.label_ids, average=AVERAGE))
    metrics.update(recall.compute(predictions=predictions, references=eval_pred.label_ids, average=AVERAGE))
    metrics.update(f1.compute(predictions=predictions, references=eval_pred.label_ids, average=AVERAGE))
    return metrics

In [18]:
# Setup the trainer
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=processed_datasets["train"],
    eval_dataset=processed_datasets["valid"],
    processing_class=feature_extractor,
    compute_metrics=compute_metrics,  # Use the metrics function from above
)

In [19]:
trainer.train()

<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: zh-preston (zh-preston-queen-s-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.001000,0.003579,0.998831,0.999057,0.999450,0.999253
2,0.001100,0.003091,0.999108,0.999725,0.999135,0.999430
3,0.002000,0.001945,0.999539,0.999646,0.999764,0.999705
4,0.000000,0.001803,0.999723,0.999803,0.999843,0.999823


TrainOutput(global_step=22855, training_loss=0.004115778539612579, metrics={'train_runtime': 27434.7446, 'train_samples_per_second': 106.649, 'train_steps_per_second': 0.833, 'total_flos': 1.9828487041060543e+20, 'train_loss': 0.004115778539612579, 'epoch': 4.998961010553946})

In [20]:
trainer.save_model(MODEL_SAVE_DIR)

In [21]:
trainer.evaluate()

{'eval_loss': 0.0018028703052550554,
 'eval_accuracy': 0.999723162103968,
 'eval_precision': 0.9998034745696093,
 'eval_recall': 0.9998427734758853,
 'eval_f1': 0.999823123636578,
 'eval_runtime': 143.347,
 'eval_samples_per_second': 226.792,
 'eval_steps_per_second': 7.088,
 'epoch': 4.998961010553946}

In [ ]:
trainer.push_to_hub()